In [1]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from beam_postgres import splitters
from beam_postgres.io import ReadFromPostgres
from datetime import datetime

class FilterWeekends(beam.DoFn):
    def process(self, element):
        # Get the datetime from the element
        dt = element['datetime']
        
        # Check if the day of the week is a weekend (5 or 6 are Saturday and Sunday)
        if dt.weekday() >= 5:
            yield element

class FormatDataFn(beam.DoFn):
    def process(self, element):
        # Format datetime to a user-friendly string
        formatted_datetime = element['datetime'].strftime('%Y-%m-%d %H:%M:%S')
        
        # Format other values as needed
        formatted_street_time = round(element['street_time'], 2)
        formatted_count = element['count']
        formatted_velocity = round(element['velocity'], 2)
        
        # Create a new dictionary with formatted values
        formatted_element = {
            'datetime': formatted_datetime,
            'street_time': formatted_street_time,
            'count': formatted_count,
            'velocity': formatted_velocity
        }
        
        yield formatted_element

with beam.Pipeline(options=PipelineOptions()) as p:
    read_from_postgres = ReadFromPostgres(
            query="SELECT * FROM public.your_table_name;",
            host="localhost",
            database="transport",
            user="postgres",
            password="postgres",
            splitter=splitters.NoSplitter()  # you can select how to split query for performance
    )

    # Read data from PostgreSQL and store in a PCollection named 'data'
    data = p | "ReadFromPostgres" >> read_from_postgres

    # Use the 'data' PCollection and filter for weekend data
    weekend_data = data | "FilterWeekends" >> beam.ParDo(FilterWeekends())

    # Format the data using the FormatDataFn transformation
    formatted_data = weekend_data | "FormatData" >> beam.ParDo(FormatDataFn())

    # Output formatted data to stdout or other downstream steps
    (
        formatted_data
        | "WriteToStdout" >> beam.Map(print)
        | "Write to file" >> beam.io.WriteToText('data/formatted_to_hive')
        # Add more pipeline steps here
    )


INFO:beam_postgres.client:Successfully execute query: EXPLAIN SELECT * FROM (SELECT * FROM public.your_table_name) as subq
INFO:beam_postgres.client:Successfully execute query: SELECT * FROM public.your_table_name


{'datetime': '2019-02-09 07:15:00', 'street_time': 1222.0, 'count': 23, 'velocity': 84.61}
